In [35]:
import time

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

import torchtext as tt
from torchtext.vocab import Vocab

import typing
from typing import List

from collections import Counter

from tqdm import tqdm_notebook as tqdm

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [5]:
def clear_punk(text):
    punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}
    punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
    '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
    '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
    '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
    '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

    for p in punct_mapping:
        text = text.replace(p, punct_mapping[p])

    for p in punct:
        text = text.replace(p,'')

    return text

In [6]:
def cleaner(word):
    word = str(word)
    word = word.lower()
    
    word = clear_punk(word)
    word = word.strip()
    return word

In [7]:
df_hinglish = pd.read_csv('../input/hinglish-english/hinglish.txt', header = None)
df_english = pd.read_csv('../input/hinglish-english/english.txt', header = None)

In [8]:
df_hinglish['Langauge'] = 1
df_english['Langauge'] = 0

In [9]:
df_train = pd.concat([df_hinglish, df_english], axis = 0)
df_train.reset_index(inplace = True)

df_train.rename( columns = {0:"Word"} , inplace = True)
df_train.drop(['index'], axis = 1, inplace = True)

In [10]:
df_train

,Word,Langauge
0,ab,1
1,ma,1
2,me,1
3,mi,1
4,na,1
...,...,...
78239,gastroenteroanastomosis,0
78240,microspectrophotometric,0
78241,pseudolamellibranchiate,0
78242,microelectrophoretically,0


In [11]:
df_valid = pd.read_csv('../input/hinglish-english/testing_data_.csv')
df_valid = df_valid.rename(columns = {'word':'Word', 'label':'Langauge'})
df_valid = df_valid.drop(['Unnamed: 0'], axis = 1)

In [12]:
df_valid

,Word,Langauge
0,berapt,0
1,schizostelic,0
2,galiyon,1
3,kah,1
4,Kalat,0
...,...,...
1918,chodo,1
1919,bhashan,1
1920,soorat,1
1921,don,1


In [14]:
def tokenize(word):
    return [c for c in word]

In [15]:
def build_vocab(data, min_freq, tokenizer):
    counter = Counter()
    for word in data:
        word = cleaner(word)
        counter.update(tokenizer((word)))
    return Vocab(counter, min_freq=min_freq , specials=( '<unk>','<pad>', '<sos>', '<eos>'))

In [16]:
vocab = build_vocab(df_train['Word'], min_freq=1, tokenizer = tokenize)
vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7feef346c510>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<sos>': 2,
             '<eos>': 3,
             'a': 4,
             'e': 5,
             'i': 6,
             'n': 7,
             'r': 8,
             's': 9,
             'o': 10,
             't': 11,
             'l': 12,
             'h': 13,
             'c': 14,
             'u': 15,
             'd': 16,
             'p': 17,
             'm': 18,
             'g': 19,
             'y': 20,
             'b': 21,
             'k': 22,
             'v': 23,
             'f': 24,
             'j': 25,
             'w': 26,
             'z': 27,
             'x': 28,
             'q': 29})

In [17]:
vocab.stoi['a']

4

In [18]:
df_train['Word'] = df_train['Word'].apply(lambda x: str(x))
(df_train['Word'].apply(len)).max()

29

# Modeling

In [19]:
class LangData(Dataset):
    def __init__(self, df, vocab, maxlen=32):
        self.df = df
        self.vocab = vocab
        self.maxlen = maxlen
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        df = self.df
        word = str(df.loc[idx, 'Word'])
        word = cleaner(word)
        tokens = tokenize(word)
        tokens = [vocab.stoi[token] for token in tokens]
        tokens += [1] * (self.maxlen - len(tokens))
        
        label = int(df.loc[idx, 'Langauge'])
        
        return np.array(tokens), label

In [53]:
class LangDetectNet(nn.Module):
    def __init__(self, vocab_size:int, embedding_dim:int, n_layers:int, hidden_dim: int):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.LSTM = nn.LSTM(input_size = embedding_dim,
                            hidden_size = hidden_dim, 
                            num_layers = n_layers, 
                            dropout=0, 
                            bidirectional=True,
                            batch_first = True)
        self.fc1 = nn.Linear(2*hidden_dim, 5)
        self.fc2 = nn.Linear(5,1)
        self.relu = nn.ReLU()
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_x: torch.TensorType):
        # shape of x: [seq_len, batch_size]
        x = self.embedding(input_x)
        #shape of x: [seq_len, batch_size, embedding_dim]
        outp, (hidden, cell) = self.LSTM(x)

        # shape of outp: [seq_len, batch_size, 2*hidden_dim]
        hidden_last = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        x = self.relu(self.fc1(hidden_last))
        x = self.fc2(x)
        x = self.sigmoid(x)

        return x

In [54]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
Progress_Bar = True

In [55]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    model.train()
    bar = tqdm(iterator) if Progress_Bar else iterator
    
    for (x, y) in bar:
        x = torch.LongTensor(x)
        
        y = y.float()
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        y_pred = torch.reshape(y_pred, (-1,))
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        loss_np = loss.detach().cpu().numpy()
        epoch_loss += loss_np
        if Progress_Bar:
            bar.set_description('Training loss: %.5f' % (loss_np))
        
    return epoch_loss/len(iterator)

def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    preds = []
    preds = np.array(preds)
    targets = []
    targets = np.array(targets)
    model.eval()
    bar = tqdm(iterator) if Progress_Bar else iterator
    
    with torch.no_grad():
        
        for (x, y) in bar:
            x = torch.LongTensor(x)
            y = y.float()
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            y_pred = torch.reshape(y_pred, (-1,))
            loss = criterion(y_pred, y)
            loss_np = loss.detach().cpu().numpy()
            epoch_loss += loss_np
            
            y_pred = y_pred.detach().cpu().numpy()
            y_pred = [1 if pred>0.5 else 0 for pred in y_pred]
            preds = np.append(preds, y_pred)
            targets = np.append(targets, y.detach().cpu().numpy())
#             preds = preds.reshape(-1)
#             targets = targets.reshape(-1)
            
            if Progress_Bar:
                bar.set_description('Validation loss: %.5f' % (loss_np))
            
            
     
    return epoch_loss/len(iterator), accuracy_score(preds, targets)

In [56]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


def fit_model(model, model_name, train_iterator, valid_iterator, optimizer, loss_criterion, device, epochs):
    best_valid_loss = float('inf')
    
    train_losses = []
    valid_losses = []
    valid_metric_scores = []
    
    for epoch in range(epochs):
    
        start_time = time.time()
    
        train_loss = train(model, train_iterator, optimizer, loss_criterion, device)
        valid_loss, valid_metric_score = evaluate(model, valid_iterator, loss_criterion, device)
        
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        valid_metric_scores.append(valid_metric_score)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), f'{model_name}.pt')
    
        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
        print(f'Epoch: {epoch+1: 2} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'Train Loss: {train_loss:.3f}')
        print(f'Val. Loss: {valid_loss:.3f} |  Val. Metric Score: {valid_metric_score:.3f}')
        
    return train_losses, valid_losses, valid_metric_scores

In [61]:
train_data = LangData(df_train, vocab)
train_loader = DataLoader(train_data, shuffle = True, batch_size = 64, num_workers = 4)

valid_data = LangData(df_valid, vocab)
valid_loader = DataLoader(valid_data, shuffle = True, batch_size = 64, num_workers = 4)

model = LangDetectNet(vocab_size = len(vocab.stoi),
                   embedding_dim = 4,
                    hidden_dim = 20,
                   n_layers = 1 ).to(device)

loss_criterion = nn.BCELoss()

opt = optim.Adam(model.parameters(), lr = 1e-2, betas=(0.9,0.999))
fit_model(model, 'LangaugeDetection',  train_loader, valid_loader, opt, loss_criterion, device, epochs = 5)

  0%|          | 0/1223 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch:  1 | Epoch Time: 0m 17s
Train Loss: 0.287
Val. Loss: 0.235 |  Val. Metric Score: 0.913


  0%|          | 0/1223 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

...

Epoch:  4 | Epoch Time: 0m 17s
Train Loss: 0.192
Val. Loss: 0.259 |  Val. Metric Score: 0.921


  0%|          | 0/1223 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch:  5 | Epoch Time: 0m 17s
Train Loss: 0.189
Val. Loss: 0.213 |  Val. Metric Score: 0.922


([0.2871102037506813,
  0.21558846225954428,
  0.19993467142200821,
  0.19213021324851526,
  0.1885560579957381],
 [0.23498515111784782,
  0.2245346966289705,
  0.220403490047301,
  0.259091361155433,
  0.21252560519403027],
 [0.9131565262610505,
  0.9167966718668746,
  0.9193967758710349,
  0.9214768590743629,
  0.921996879875195])

In [62]:
opt = optim.Adam(model.parameters(), lr = 1e-3, betas=(0.9,0.999))

fit_model(model, 'LangaugeDetection',  train_loader, valid_loader, opt, loss_criterion, device, epochs = 10)

  0%|          | 0/1223 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch:  1 | Epoch Time: 0m 16s
Train Loss: 0.176
Val. Loss: 0.206 |  Val. Metric Score: 0.925


  0%|          | 0/1223 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

...

Epoch:  9 | Epoch Time: 0m 17s
Train Loss: 0.164
Val. Loss: 0.196 |  Val. Metric Score: 0.924


  0%|          | 0/1223 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch:  10 | Epoch Time: 0m 17s
Train Loss: 0.164
Val. Loss: 0.197 |  Val. Metric Score: 0.926


([0.17568946340085537,
  0.17134613169696813,
  0.16970685289187307,
  0.1682222550346509,
  0.16740788086846728,
  0.166593471683641,
  0.16559981905801172,
  0.16503706107484153,
  0.1644335581402455,
  0.16379763349549553],
 [0.20588486737781955,
  0.20214012637734413,
  0.20406427691059728,
  0.20506888964483816,
  0.21449025552118978,
  0.2004180745251717,
  0.1972419249434625,
  0.1959010981503994,
  0.19621096347128192,
  0.19654222313434846],
 [0.9251170046801872,
  0.9225169006760271,
  0.9230369214768591,
  0.9230369214768591,
  0.9240769630785232,
  0.9240769630785232,
  0.9251170046801872,
  0.9240769630785232,
  0.9240769630785232,
  0.9256370254810192])

In [63]:
opt = optim.Adam(model.parameters(), lr = 1e-1, betas=(0.9,0.999))
fit_model(model, 'LangaugeDetection',  train_loader, valid_loader, opt, loss_criterion, device, epochs = 2)

  0%|          | 0/1223 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch:  1 | Epoch Time: 0m 17s
Train Loss: 0.321
Val. Loss: 0.430 |  Val. Metric Score: 0.816


  0%|          | 0/1223 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch:  2 | Epoch Time: 0m 17s
Train Loss: 0.326
Val. Loss: 0.383 |  Val. Metric Score: 0.830


([0.3211090077546319, 0.32598075868892357],
 [0.4303375713286861, 0.3827828928347557],
 [0.8164326573062922, 0.8299531981279251])

In [65]:
opt = optim.Adam(model.parameters(), lr = 1e-6, betas=(0.9,0.999))
fit_model(model, 'LangaugeDetection',  train_loader, valid_loader, opt, loss_criterion, device, epochs = 100)

  0%|          | 0/1223 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch:  1 | Epoch Time: 0m 18s
Train Loss: 0.316
Val. Loss: 0.342 |  Val. Metric Score: 0.852


  0%|          | 0/1223 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

...

Epoch:  99 | Epoch Time: 0m 17s
Train Loss: 0.309
Val. Loss: 0.373 |  Val. Metric Score: 0.848


  0%|          | 0/1223 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch:  100 | Epoch Time: 0m 17s
Train Loss: 0.309
Val. Loss: 0.347 |  Val. Metric Score: 0.848


([0.31579130118558957,
  0.31557769255860363,
  0.31528808308105827,
  0.3150043518252774,
  0.3147861918804593,
  0.31458448987219634,
  0.3143389719071587,
  0.3141782469712411,
  0.31386695012681715,
  0.3137738612082194,
  0.31350900638522616,
  0.3133302427834404,
  0.3132112267581732,
  0.3131085109523058,
  0.3127955297953948,
  0.31277553413066334,
  0.31255876360642354,
  0.3124394626132017,
  0.3122733436518776,
  0.312147210780851,
  0.31209066956138765,
  0.311975951832723,
  0.31186866199785046,
  0.3117204569493369,
  0.3116608445920227,
  0.31149949587254083,
  0.31144585012049336,
  0.31136519115232386,
  0.3112604266987506,
  0.3111919232949505,
  0.31115268326180934,
  0.31112603831028024,
  0.31104534397774414,
  0.31090797705671774,
  0.3108710123626599,
  0.3107997838617931,
  0.3106981513082055,
  0.3107576993639023,
  0.3106001924756893,
  0.3105528429455184,
  0.310536762716238,
  0.310437075543969,
  0.31040519403441463,
  0.3103607624452424,
  0.31024955436039

In [80]:
em

tensor([[ 1.1929,  1.1133, -0.7558],
        [-0.0497,  0.1209,  2.5164],
        [-0.7330,  1.0676, -0.9761]], grad_fn=<EmbeddingBackward>)